In [10]:
# # If needed to convert MP3 into WAV
# from pydub import AudioSegment
# audio = AudioSegment.from_mp3("hey_siri.mp3")
# audio.export("hey_siri.wav", format="wav")

In [11]:
# Imports
import numpy as np
from scipy.io.wavfile import read, write
from scipy.signal import butter, filtfilt
import csv

In [12]:
# Load Audio FIle
fs, audio_signal = read("hey_siri.wav")

In [13]:
# Make Sure Audio Is Mono
if len(audio_signal.shape) > 1:
    audio_signal = audio_signal[:, 0]

In [14]:
# Normalize Audio
audio_signal = audio_signal / np.max(np.abs(audio_signal))

In [15]:
# Generate High-Frequency Carrier
duration = len(audio_signal) / fs
t = np.linspace(0, duration, len(audio_signal), endpoint=False)
carrier_frequency = 20000
carrier_wave = np.sin(2 * np.pi * carrier_frequency * t)

In [16]:
# High Pass Filter
def high_pass_filter(signal, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return filtfilt(b, a, signal)

In [17]:
# Modulate
modulated_signal = audio_signal * carrier_wave
modulated_signal = high_pass_filter(modulated_signal, cutoff=20000, fs=fs)
modulated_signal = modulated_signal / np.max(np.abs(modulated_signal)) * 0.9 # Boost Amplitude

In [18]:
# Write Output File
write("modulated_audio.wav", fs, (modulated_signal * 32767).astype(np.int16))